# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room Part #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating a simple RAG pipeline with [LangChain v0.2.0](https://python.langchain.com/v0.2/docs/versions/v0_2/)
  4. Synthetic Dataset Generation for Evaluation using the [Ragas](https://github.com/explodinggradients/ragas) framework.
  

- 🤝 Breakout Room Part #2:
  1. Evaluating our pipeline with Ragas
  3. Making Adjustments to our RAG Pipeline
  4. Evaluating our Adjusted pipeline against our baseline
  5. Testing OpenAI's Claim

The only way to get started is to get started - so let's grab our dependencies for the day!

> NOTE: Using this notebook as presented will occur a charge of ~$3USD from OpenAI usage. Most of this cost is produced by the Synthetic Data Generation step - if you want to reduce costs, please use the provided commented code to leverage `GPT-3.5-Turbo` as the `critic_llm`!

## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

# 🤝 Breakout Room Part #1

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://blog.langchain.dev/langchain-v0-1-0/) of LangChain v0.1.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [1]:
# !pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai

We'll also get the "star of the show" today, which is Ragas!

In [2]:
# !pip install -qU ragas

We'll be leveraging [QDrant](https://qdrant.tech/) again as our LangChain `VectorStore`.

We'll also install `pymupdf` and its dependencies which will allow us to load PDFs using the `PyMuPDFLoader` in the `langchain-community` package!

In [3]:
# !pip install -qU qdrant-client pymupdf pandas

## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [4]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

## Task 3: Creating a Simple RAG Pipeline with LangChain v0.1.0

Building on what we learned last week, we'll be leveraging LangChain v0.1.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.2.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [5]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

documents = loader.load()

In [6]:
documents[0].metadata

{'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'page': 0,
 'total_pages': 195,
 'format': 'PDF 1.3',
 'title': 'The Pmarca Blog Archives',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': 'Mac OS X 10.10 Quartz PDFContext',
 'creationDate': "D:20150110020418Z00'00'",
 'modDate': "D:20150110020418Z00'00'",
 'trapped': ''}

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

Let's confirm we've split our document.

In [8]:
len(documents)

1864

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

In [9]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

#### Creating a QDrant VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

In [10]:
from langchain_community.vectorstores import Qdrant

qdrant_vector_store = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="PMarca Blogs",
)

####❓ Question #1:

List out a few of the techniques that Qdrant uses that make it performant.

> NOTE: Check the [documentation](https://qdrant.tech/documentation/overview/) for more information about QDrant!

    `1. Approximate Nearest Neighbor (ANN) search for fast simlarity queries on high-dim vectors`

    `2. Offers optimized storage options in-memory (store in RAM) and memmap (virtual address on-disk) for balancing performance and usage`

    `3. Optimized indexing particularly for sparse vectors`

    `4. Allows control over allocating CPU threads for indexing, enabling for index building`

    `5. Allows segregation of storage and compute components leading to optimized perfomance by having dedicated resources`

    `6. Allows for similarity search using various distance metrics` 

#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [11]:
retriever = qdrant_vector_store.as_retriever()

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [12]:
retrieved_documents = retriever.invoke("What is a rule of thumb for selecting an industry to invest in?")

In [13]:
for doc in retrieved_documents:
  print(doc)

page_content='the existing order — and make sure that those forces of change\nhave a reasonable chance at succeeding.\nSecond rule of thumb:\nOnce you have picked an industry, get right to the center of it' metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 125, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': 'e29af39df9c3469981bbda033e085b53', '_collection_name': 'PMarca Blogs'}
page_content='Third rule:\nIn a rapidly changing Held like technology, the best place to\nget experience when you’re starting out is in younger, high-\ngrowth companies.' metadata={'source': 'https://d1lamhf6l6yk6d

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [14]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [15]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [16]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [17]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

####🏗️ Activity #1:

Describe the pipeline shown above in simple terms. You can include a diagram if desired.

In [18]:
print(retrieval_augmented_qa_chain.get_graph().draw_ascii())

                       +---------------------------------+                         
                       | Parallel<context,question>Input |                         
                       +---------------------------------+                         
                           *****                   ****                            
                        ***                            ****                        
                     ***                                   ****                    
+--------------------------------+                             **                  
| Lambda(itemgetter('question')) |                              *                  
+--------------------------------+                              *                  
                 *                                              *                  
                 *                                              *                  
                 *                                              *           

Let's test it out!

In [19]:
question = "What is a rule of thumb for selecting an industry to invest in?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

Get right to the center of it.


In [20]:
question = "What did Pink Floyd have to say about how to proceed when investing in a new industry?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

I don't know.
[Document(page_content='ask if you can call them again if things change.\nTrust me — they’d much rather be saying “yes” than “no” —\nthey need all the good investments they can get.\nSecond, consider the environment.', metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 15, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': 'b233e4d1d5b44d6a9608b13624305b22', '_collection_name': 'PMarca Blogs'}), Document(page_content='watching carefully — if everyone agrees right up front that\nwhatever you are doing makes total sense, it probably isn’t a new\nand radical enough idea to justify a

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

## Task 4: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evaluating on every core metric today, but in order to do that - we'll need to create a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

In [21]:
loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

eval_documents = loader.load()

text_splitter_eval = RecursiveCharacterTextSplitter(
    chunk_size = 600, # versus LCEL chunk_size = 200
    chunk_overlap = 50
)

eval_documents = text_splitter_eval.split_documents(eval_documents)

####❓ Question #2:

Why is it important to split our documents using different parameters when creating our synthetic data? 

`1. For increased data sample diversity and subsequent improved model performance via generalization to reaL-world scenarios`

`2. Help the synthetic dataset cover edge cases which ere overlooked using the base splitting parameters`

`3. Reduced bias in the evaluation process by capturing a different data distribution of the same sample dataset`

In [22]:
len(eval_documents)

624


> NOTE: 🛑 Using this notebook as presented will occur a charge of ~$3USD from OpenAI usage. Most of this cost is produced by the Synthetic Data Generation step - if you want to reduce costs, please use the provided commented code to leverage GPT-3.5-Turbo as the critic_llm. If you're attempting to create a lot of samples please be aware of cost, as well as rate limits. 🛑

In [23]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
# critic_llm = ChatOpenAI(model="gpt-3.5-turbo") <--- If you don't have GPT-4 access, or to reduce cost/rate limiting issues.
critic_llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

testset = generator.generate_with_langchain_docs(eval_documents, 20, distributions, is_async = False)
testset.to_pandas()

embedding nodes:   0%|          | 0/1248 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/20 [00:00<?, ?it/s]

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is the best way to develop contacts with ...,[and identifying those VCs and screening out a...,"The best way to develop contacts with VCs, in ...",simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
1,How can CEOs ensure the success of their compa...,"[strong, sharp, aggressive, ambitious director...",CEOs can ensure the success of their company b...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
2,What is the benefit of combining a useful grad...,[workforce in a high-impact way when you gradu...,"You become a ""double threat"" by combining a us...",simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
3,How does the Liking/Loving Tendency lead indiv...,[One very practical consequence of Liking/Lovi...,The Liking/Loving Tendency leads individuals t...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
4,What are some lessons learned on how to hire t...,[How to hire the best people you've\never work...,"There are many aspects to hiring great people,...",simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
5,How does the Doubt-Avoidance Tendency contribu...,"[chains I forged in life,” he is talking about...","A quickly reached conclusion, triggered by Dou...",simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
6,What industry is Washington DC known for in te...,[most interesting opportunity available — the ...,Washington DC is known for the politics industry.,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
7,What are some strategies for undergrads to gai...,"[undergrads to do some of the work, and being ...",aggressively pursue internship and co-op progr...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
8,What is the importance of aggressively seeking...,"[developing a large market, as opposed to Xght...",It is much more important for a startup to agg...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
9,Are the anticipated career peaks for creators ...,[These three components are conspicuously link...,nan,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True


####❓ Question #3:

`{simple: 0.5, reasoning: 0.25, multi_context: 0.25}`

What exactly does this mapping refer to? 

> NOTE: Check out the Ragas documentation on this generation process [here](https://docs.ragas.io/en/stable/concepts/testset_generation.html).

`This distribution lists the manner in which sysnthetic question-context-ground_truth triplets are generated using an llm and allocating specific rations for the type of questions generated:`

`- Reasoning:  Rewrite the question in a way that enhances the need for reasoning to answer it effectively.`

`- Multi-Context: Rephrase the question in a manner that necessitates information from multiple related sections or chunks to formulate an answer.`

`- Simple: Straightforward Q-C pairs which are basic factual questions that can be directly answered from the given document content without requiring complex reasoning or multiple context pieces.`

Let's look at the output and see what we can learn about it!

In [24]:
testset.test_data[0]

DataRow(question='What is the best way to develop contacts with VCs?', contexts=['and identifying those VCs and screening out all the others is\nabsolutely key.\nNow, on to developing contacts\nThe best way to develop contacts with VCs, in my opinion, is to\nwork at a venture-backed startup, kick butt, get promoted, and\nnetwork the whole way.\n20\nThe Pmarca Blog Archives'], ground_truth='The best way to develop contacts with VCs, in my opinion, is to work at a venture-backed startup, kick butt, get promoted, and network the whole way.', evolution_type='simple', metadata=[{'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 24, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:2015011

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [25]:
test_df = testset.to_pandas()

In [26]:
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is the best way to develop contacts with ...,[and identifying those VCs and screening out a...,"The best way to develop contacts with VCs, in ...",simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
1,How can CEOs ensure the success of their compa...,"[strong, sharp, aggressive, ambitious director...",CEOs can ensure the success of their company b...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
2,What is the benefit of combining a useful grad...,[workforce in a high-impact way when you gradu...,"You become a ""double threat"" by combining a us...",simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
3,How does the Liking/Loving Tendency lead indiv...,[One very practical consequence of Liking/Lovi...,The Liking/Loving Tendency leads individuals t...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
4,What are some lessons learned on how to hire t...,[How to hire the best people you've\never work...,"There are many aspects to hiring great people,...",simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
5,How does the Doubt-Avoidance Tendency contribu...,"[chains I forged in life,” he is talking about...","A quickly reached conclusion, triggered by Dou...",simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
6,What industry is Washington DC known for in te...,[most interesting opportunity available — the ...,Washington DC is known for the politics industry.,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
7,What are some strategies for undergrads to gai...,"[undergrads to do some of the work, and being ...",aggressively pursue internship and co-op progr...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
8,What is the importance of aggressively seeking...,"[developing a large market, as opposed to Xght...",It is much more important for a startup to agg...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
9,Are the anticipated career peaks for creators ...,[These three components are conspicuously link...,nan,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True


In [27]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [28]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [29]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's take a peek and see what that looks like!

In [30]:
response_dataset[0]

{'question': 'What is the best way to develop contacts with VCs?',
 'answer': 'The best way to develop contacts with VCs is to work at a venture-backed startup, excel, get promoted, and network along the way.',
 'contexts': ['absolutely key.\nNow, on to developing contacts\nThe best way to develop contacts with VCs, in my opinion, is to\nwork at a venture-backed startup, kick butt, get promoted, and\nnetwork the whole way.\n20',
  'If\nyou\nengage\nin\na\nset\nof\nthese\ntechniques\nover\ntime,\nyou should be able to interact with at least a few VCs in ways that\nthey Xnd useful and that might lead to further conversations',
  'At best, a VC blogger may encourage her readers to communi-\ncate with her in various ways, including soliciting email pitches\nin certain startup categories of interest to her.',
  'in appropriate ways.\nAt the very least you will start to get a really good sense of which\nVCs who blog are interested in which kinds of companies.'],
 'ground_truth': 'The best wa

# 🤝 Breakout Room Part #2

## Task 1: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [31]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

All that's left to do is call "evaluate" and away we go!

In [32]:
results = evaluate(response_dataset, metrics)

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

No statements were generated from the answer.
No statements were generated from the answer.
No statements were generated from the answer.


In [33]:
results

{'faithfulness': 0.7700, 'answer_relevancy': 0.7940, 'context_recall': 0.7333, 'context_precision': 0.8347, 'answer_correctness': 0.5077}

In [34]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What is the best way to develop contacts with ...,The best way to develop contacts with VCs is t...,"[absolutely key.\nNow, on to developing contac...","The best way to develop contacts with VCs, in ...",1.000000,0.999998,1.000000,1.000000,0.429184
1,How can CEOs ensure the success of their compa...,CEOs can ensure the success of their company b...,[Then develop a list of your top 20 or 30 up a...,CEOs can ensure the success of their company b...,0.500000,0.934325,0.500000,1.000000,0.429673
2,What is the benefit of combining a useful grad...,"The benefit is becoming a ""double threat.""","[degree, you are much better oW combining it w...","You become a ""double threat"" by combining a us...",1.000000,0.981204,1.000000,0.833333,0.718952
3,How does the Liking/Loving Tendency lead indiv...,The Liking/Loving Tendency leads individuals t...,[One very practical consequence of Liking/Lovi...,The Liking/Loving Tendency leads individuals t...,1.000000,0.891585,1.000000,1.000000,0.543909
4,What are some lessons learned on how to hire t...,Some lessons learned on how to hire the best p...,[How to hire the best people you've\never work...,"There are many aspects to hiring great people,...",0.500000,1.000000,1.000000,1.000000,0.224064
5,How does the Doubt-Avoidance Tendency contribu...,The Doubt-Avoidance Tendency contributes to er...,[Four: Doubt-Avoidance Tendency\nThe brain of ...,"A quickly reached conclusion, triggered by Dou...",1.000000,0.915922,1.000000,0.805556,0.542137
6,What industry is Washington DC known for in te...,Politics,[the city where all the action is happening.\n...,Washington DC is known for the politics industry.,NaN,0.856296,1.000000,1.000000,0.206403
7,What are some strategies for undergrads to gai...,Some strategies for undergrads to gain real-wo...,[What should I do while I’m in school?\nI’m a ...,aggressively pursue internship and co-op progr...,1.000000,0.989371,1.000000,1.000000,0.635767
8,What is the importance of aggressively seeking...,The importance of aggressively seeking out a b...,"[case, your enemy is the world at large.)\nIt ...",It is much more important for a startup to agg...,1.000000,0.886707,0.500000,0.500000,0.532059
9,Are the anticipated career peaks for creators ...,Earlier,"[early, end late, and produce at above-average...",nan,NaN,0.000000,0.800000,0.833333,0.953437


## Task 2: Making Adjustments to our RAG Pipeline

Now that we have established a baseline - we can see how any changes impact our pipeline's performance!

Let's modify our retriever and see how that impacts our Ragas metrics!

> NOTE: MultiQueryRetriever is expanded on [here](https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever) but for now, the implementation is not important to our lesson!

In [35]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=primary_qa_llm)

We'll also re-create our RAG pipeline using the abstractions that come packaged with LangChain v0.1.0!

First, let's create a chain to "stuff" our documents into our context!

In [36]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

Next, we'll create the retrieval chain!

In [37]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [38]:
response = retrieval_chain.invoke({"input": "Who is Taylor Swift fueding with?"})

In [39]:
print(response["answer"])

I'm sorry, but based on the context provided, I do not have any information about Taylor Swift or any feud she may be involved in.


In [40]:
response = retrieval_chain.invoke({"input": "Why are they fueding?"})

In [41]:
print(response["answer"])

The text does not provide any information about a feud or conflict between individuals or groups. It mainly discusses human tendencies towards dislike and hate, the history of war, factors contributing to success in startups, and the shift of consumer attention towards online media.


In [45]:
response = retrieval_chain.invoke({"input": "What did Pink Floyd have to say about how to proceed when investing in a new industry?"})
print(response["answer"])

Pink Floyd advised to "set the controls for the heart of the sun" when investing in a new industry. This suggests that one should make sure they are heading towards where the action is, where the disruptive change and greater opportunity lie.


Well, just from those responses this chain *feels* better - but lets see how it performs on our eval!

Let's do the same process we did before to collect our pipeline's contexts and answers.

In [44]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

Now we can convert this into a dataset, just like we did before.

In [46]:
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [52]:
try:
    advanced_retrieval_results = evaluate(
        response_dataset_advanced_retrieval, metrics, raise_exceptions=False
    )
except Exception as e:
    print(f"Exception caught: {e}")


Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

Let's evaluate on the same metrics we did for the first pipeline and see how it does!

In [53]:
advanced_retrieval_results_df = advanced_retrieval_results.to_pandas()
advanced_retrieval_results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What is the best way to develop contacts with ...,The best way to develop contacts with VCs is t...,[If\nyou\nengage\nin\na\nset\nof\nthese\ntechn...,"The best way to develop contacts with VCs, in ...",1.000000,1.000000,1.0,0.500000,0.455156
1,How can CEOs ensure the success of their compa...,CEOs can ensure the success of their company b...,[Then develop a list of your top 20 or 30 up a...,CEOs can ensure the success of their company b...,0.500000,0.980001,0.5,0.976190,0.353037
2,What is the benefit of combining a useful grad...,Combining a useful graduate degree with a subs...,"[degree, you are much better oW combining it w...","You become a ""double threat"" by combining a us...",1.000000,0.801313,1.0,1.000000,0.537089
3,How does the Liking/Loving Tendency lead indiv...,The Liking/Loving Tendency leads individuals t...,[One very practical consequence of Liking/Lovi...,The Liking/Loving Tendency leads individuals t...,1.000000,0.902801,1.0,1.000000,0.456037
4,What are some lessons learned on how to hire t...,Some lessons learned on how to hire the best p...,[How to hire the best people you've\never work...,"There are many aspects to hiring great people,...",0.600000,1.000000,1.0,1.000000,0.217531
5,How does the Doubt-Avoidance Tendency contribu...,The Doubt-Avoidance Tendency contributes to er...,[Four: Doubt-Avoidance Tendency\nThe brain of ...,"A quickly reached conclusion, triggered by Dou...",1.000000,0.930578,1.0,1.000000,0.906889
6,What industry is Washington DC known for in te...,Washington DC is known for the politics industry.,[the city where all the action is happening.\n...,Washington DC is known for the politics industry.,1.000000,0.925713,1.0,0.666667,1.000000
7,What are some strategies for undergrads to gai...,Some strategies for undergrads to gain real-wo...,[What should I do while I’m in school?\nI’m a ...,aggressively pursue internship and co-op progr...,1.000000,1.000000,1.0,0.916667,0.728576
8,What is the importance of aggressively seeking...,Aggressively seeking out a big market is impor...,[the most important factor in a startup’s succ...,It is much more important for a startup to agg...,1.000000,0.985946,0.5,0.810000,0.449072
9,Are the anticipated career peaks for creators ...,The anticipated career peaks for creators who ...,"[early, end late, and produce at above-average...",nan,1.000000,0.982765,0.0,0.805556,0.184018


## Task 3: Evaluating our Adjusted Pipeline Against Our Baseline

Now we can compare our results and see what directional changes occured!

Let's refresh with our initial metrics.

In [73]:
results

{'faithfulness': 0.7700, 'answer_relevancy': 0.7940, 'context_recall': 0.7333, 'context_precision': 0.8347, 'answer_correctness': 0.5077}

And see how our advanced retrieval modified our chain!

In [74]:
advanced_retrieval_results

{'faithfulness': 0.7488, 'answer_relevancy': 0.9406, 'context_recall': 0.6600, 'context_precision': 0.8598, 'answer_correctness': 0.4815}

In [75]:
import pandas as pd

df_original = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_comparison = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'MultiQueryRetriever with Document Stuffing'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')

df_merged['Delta'] = df_merged['MultiQueryRetriever with Document Stuffing'] - df_merged['Baseline']

df_merged

,Metric,Baseline,MultiQueryRetriever with Document Stuffing,Delta
0,faithfulness,0.770028,0.748750,-0.021278
1,answer_relevancy,0.793998,0.940624,0.146626
2,context_recall,0.733333,0.660000,-0.073333
3,context_precision,0.834722,0.859790,0.025067
4,answer_correctness,0.507744,0.481545,-0.026199


## Task 4: Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #2:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

*Using the OpenAI recommended `text-embedding-3-small` embedding model*

In [78]:
new_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

*Storing document chunks in Qdrant vector store as embeddings generated from the new embedding model above*

In [79]:
vector_store = Qdrant.from_documents(
    documents,
    new_embeddings,
    location=":memory:",
    collection_name="PMarca Blogs - TE3 - MQR",
)

*Setup the Qdrant retriever*

In [80]:
new_retriever = vector_store.as_retriever()

*Setup the advanced MultiQueryRetriever using the primary OpenAI llm*

In [81]:
new_advanced_retriever = MultiQueryRetriever.from_llm(retriever=new_retriever, llm=primary_qa_llm)

*Setup the retireval chain using the advanced retriever & context stuffing chain developed earlier*

In [82]:
new_retrieval_chain = create_retrieval_chain(new_advanced_retriever, document_chain)

*Generate answers and fetch context using the new embedding model & primary llm from the synthetic questions/ground truth dataset*

In [83]:
answers = []
contexts = []

for question in test_questions:
  response = new_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

In [84]:
new_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [89]:
try:
    new_advanced_retrieval_results = evaluate(
        new_response_dataset_advanced_retrieval, metrics, raise_exceptions=False
    )
except Exception as e:
    print(f"Exception caught: {e}")

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

In [90]:
new_advanced_retrieval_results

{'faithfulness': 0.8881, 'answer_relevancy': 0.9447, 'context_recall': 0.7242, 'context_precision': 0.8581, 'answer_correctness': 0.5387}

*Comparison of Evals - Baseline v Advanced MultiQuery Retrieval v Advanced Retrieval + Improved Embedding Model*

In [91]:
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'ADA + Baseline'])
df_original = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'ADA + MQR'])
df_comparison = pd.DataFrame(list(new_advanced_retrieval_results.items()), columns=['Metric', 'TE3 + MQR'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")

df_merged['ADA + MQR -> TE3 + MQR'] = df_merged['TE3 + MQR'] - df_merged['ADA + MQR']
df_merged['Baseline -> TE3 + MQR'] = df_merged['TE3 + MQR'] - df_merged['ADA + Baseline']

df_merged

,Metric,ADA + Baseline,ADA + MQR,TE3 + MQR,ADA + MQR -> TE3 + MQR,Baseline -> TE3 + MQR
0,faithfulness,0.770028,0.748750,0.888095,0.139345,0.118067
1,answer_relevancy,0.793998,0.940624,0.944682,0.004058,0.150684
2,context_recall,0.733333,0.660000,0.724167,0.064167,-0.009167
3,context_precision,0.834722,0.859790,0.858056,-0.001734,0.023333
4,answer_correctness,0.507744,0.481545,0.538672,0.057127,0.030927


####❓ Question #4:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`?

## BONUS ACTIVITY: Using a Better Generator

## Task 5: Using a better generator model

Now that we've seen how much more effective a better Retrieval pipeline is, let's look at what impact a better(?) Generator is!

Adapt the above `TE3 + MQR` pipeline to use `GPT-4o` and compare the results below!

In [92]:
new_primary_qa_llm = ChatOpenAI(model="gpt-4o")

In [93]:
new_advanced_retriever_and_model = MultiQueryRetriever.from_llm(retriever=new_retriever, llm=new_primary_qa_llm)

*Setup the retireval chain using the advanced retriever & context stuffing chain developed earlier*

In [94]:
new_model_retrieval_chain = create_retrieval_chain(new_advanced_retriever_and_model, document_chain)

*Generate answers and fetch context using the new embedding model & primary llm from the synthetic questions/ground truth dataset*

In [95]:
answers = []
contexts = []

for question in test_questions:
  response = new_model_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

In [96]:
new_response_dataset_advanced_retrieval_and_model = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [ ]:
try:
    new_advanced_retrieval_and_model_results = evaluate(
        new_response_dataset_advanced_retrieval_and_model, metrics, raise_exceptions=False
    )
except Exception as e:
    print(f"Exception caught: {e}")

In [98]:
new_advanced_retrieval_and_model_results

{'faithfulness': 0.8655, 'answer_relevancy': 0.7082, 'context_recall': 0.5069, 'context_precision': 0.8826, 'answer_correctness': 0.6845}

*Final Comparison of Evals - Baseline v Advanced MultiQuery Retrieval v Advanced Retrieval + Improved Embedding Model v Advanced Retrieval + Improved Embedding Model + Improved Model*

In [102]:
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'ADA + Baseline'])
df_original = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'ADA + MQR'])
df_comparison = pd.DataFrame(list(new_advanced_retrieval_results.items()), columns=['Metric', 'TE3 + MQR'])
df_comparison_2 = pd.DataFrame(list(new_advanced_retrieval_and_model_results.items()), columns=['Metric', 'TE3 + MQR + GPT-4o'])

df_merged = pd.merge(df_comparison, df_comparison_2, on='Metric')
df_merged = pd.merge(df_original, df_merged, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")

df_merged['ADA + MQR -> TE3 + MQR'] = df_merged['TE3 + MQR'] - df_merged['ADA + MQR']
df_merged['Baseline -> TE3 + MQR'] = df_merged['TE3 + MQR'] - df_merged['ADA + Baseline']
df_merged['TE3 + MQR -> TE3 + MQR + GPT-4o'] = df_merged['TE3 + MQR + GPT-4o'] - df_merged['TE3 + MQR']
df_merged['Baseline -> TE3 + MQR + GPT-4o'] = df_merged['TE3 + MQR + GPT-4o'] - df_merged['ADA + Baseline']

df_merged

,Metric,ADA + Baseline,ADA + MQR,TE3 + MQR,TE3 + MQR + GPT-4o,ADA + MQR -> TE3 + MQR,Baseline -> TE3 + MQR,TE3 + MQR -> TE3 + MQR + GPT-4o,Baseline -> TE3 + MQR + GPT-4o
0,faithfulness,0.770028,0.748750,0.888095,0.865500,0.139345,0.118067,-0.022596,0.095472
1,answer_relevancy,0.793998,0.940624,0.944682,0.708177,0.004058,0.150684,-0.236505,-0.085821
2,context_recall,0.733333,0.660000,0.724167,0.506866,0.064167,-0.009167,-0.217300,-0.226467
3,context_precision,0.834722,0.859790,0.858056,0.882623,-0.001734,0.023333,0.024568,0.047901
4,answer_correctness,0.507744,0.481545,0.538672,0.684537,0.057127,0.030927,0.145866,0.176793
